In [1]:
import pandas as pd

df = pd.read_csv("../data/healthcare-dataset-stroke-data.csv")
df.head()


,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [2]:
# cleaning the data slightly before saving into database
df = df.dropna(subset=["bmi"]).copy()
df = df.reset_index(drop=True)
df.shape

(4909, 12)

In [3]:
# creating the sqlite database file
import sqlite3

conn = sqlite3.connect("../data/stroke.db")
cursor = conn.cursor()
"database ready"

'database ready'

In [4]:
# creating the required normalized tables
cursor.execute("""
CREATE TABLE IF NOT EXISTS patients (
    patient_id INTEGER PRIMARY KEY,
    gender TEXT,
    age REAL,
    ever_married TEXT,
    Residence_type TEXT,
    stroke INTEGER
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS medical_conditions (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    patient_id INTEGER,
    hypertension INTEGER,
    heart_disease INTEGER,
    avg_glucose_level REAL,
    bmi REAL,
    FOREIGN KEY(patient_id) REFERENCES patients(patient_id)
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS lifestyle (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    patient_id INTEGER,
    work_type TEXT,
    smoking_status TEXT,
    FOREIGN KEY(patient_id) REFERENCES patients(patient_id)
)
""")

conn.commit()
"tables created"


'tables created'

In [5]:
# inserting all rows into the three tables
for _, row in df.iterrows():
    patient_id = int(row["id"])

    cursor.execute("""
        INSERT OR REPLACE INTO patients 
        (patient_id, gender, age, ever_married, Residence_type, stroke)
        VALUES (?, ?, ?, ?, ?, ?)
    """, (
        patient_id,
        row["gender"],
        float(row["age"]),
        row["ever_married"],
        row["Residence_type"],
        int(row["stroke"])
    ))

    cursor.execute("""
        INSERT INTO medical_conditions
        (patient_id, hypertension, heart_disease, avg_glucose_level, bmi)
        VALUES (?, ?, ?, ?, ?)
    """, (
        patient_id,
        int(row["hypertension"]),
        int(row["heart_disease"]),
        float(row["avg_glucose_level"]),
        float(row["bmi"])
    ))

    cursor.execute("""
        INSERT INTO lifestyle
        (patient_id, work_type, smoking_status)
        VALUES (?, ?, ?)
    """, (
        patient_id,
        row["work_type"],
        row["smoking_status"]
    ))

conn.commit()
"data inserted"


'data inserted'

In [6]:
# checking if data is correctly stored in database
check_df = pd.read_sql_query("""
SELECT *
FROM patients
JOIN medical_conditions USING(patient_id)
JOIN lifestyle USING(patient_id)
LIMIT 5
""", conn)

check_df

,patient_id,gender,age,ever_married,Residence_type,stroke,id,hypertension,heart_disease,avg_glucose_level,bmi,id,work_type,smoking_status
0,9046,Male,67.0,Yes,Urban,1,1,0,1,228.69,36.6,1,Private,formerly smoked
1,31112,Male,80.0,Yes,Rural,1,2,0,1,105.92,32.5,2,Private,never smoked
2,60182,Female,49.0,Yes,Urban,1,3,0,0,171.23,34.4,3,Private,smokes
3,1665,Female,79.0,Yes,Rural,1,4,1,0,174.12,24.0,4,Self-employed,never smoked
4,56669,Male,81.0,Yes,Urban,1,5,0,0,186.21,29.0,5,Private,formerly smoked
